## Import libraries 

In [ ]:
import os 
import numpy as np
import mne
import matplotlib
from mne.preprocessing import compute_current_source_density 

In [ ]:
matplotlib.use('Qt5Agg')

### Load data

In [ ]:
# define path
sample_data_folder = '/Users/christinadelta/datasets/eeg_testing_data'
sample_raw = os.path.join(sample_data_folder, 'MEG', 'data', 'sample_audvis_raw.fif')

In [ ]:
raw = mne.io.read_raw_fif(sample_raw, preload=True)

### Choose the data 
This is a MEG recording that also contains EEG data. We will only use the EEG data, so get rid of the MEG channels

We will create a raw class/object for our raw data. To keep only the eeg data we will use the ```.pick_types()``` function

In [ ]:
raw.pick_types(eeg=True, meg=False, stim=True, eog=True)

In [ ]:
raw.plot()

### Plot the power spectrum 

In [ ]:
raw.plot_psd(fmin=1, fmax=20, n_fft=2**10, spatial_colors=True)

In [ ]:
raw.plot(duration=5, n_channels=30, show=True)

### Run Independent Component Analysis 

In [ ]:
ica = mne.preprocessing.ICA(n_components=10, random_state=97, max_iter=800)

In [ ]:
ica.fit(raw)
ica.plot_components()

### Make copy of the original raw, apply ICA and visualise both

In [ ]:
orig_raw = raw.copy()
ica.apply(raw)

In [ ]:
orig_raw.plot(start=0, duration=5, block=False, title= 'before ICA')
raw.plot(start=0, duration=5, block=True, title='after ICA')

### Add events

In [ ]:
events = mne.find_events(raw)
print(events[:5])

In [ ]:
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}

In [ ]:
events_fig = mne.viz.plot_events(events, event_id=event_id,  sfreq=raw.info['sfreq'], 
                          first_samp=raw.first_samp)

### Epoch the data

In [ ]:
reject_criteria = dict(eeg=150e-6,
                       eog=250e-6 )

In [ ]:
epochs = mne.Epochs(raw, events, event_id=event_id, tmin=-0.2, tmax=0.5, 
                    reject=reject_criteria, preload=True)

In [ ]:
epochs.plot()

#### Equalise the number of trials for each condition

In [ ]:
epochs.equalize_event_counts(['Auditory/Left', 'Auditory/Right', 'Visual/Left', 'Visual/Right'])
aud_epochs = epochs['Auditory']
vis_epochs = epochs['Visual']
left_epochs = epochs['Left']
right_epochs = epochs['Right']

#### Plot individual trials 

In [ ]:
aud_epochs.plot_image(picks=['EEG 021'])
vis_epochs.plot_image(picks=['EEG 059'])

### Time-Frequency Analysis of the auditory epochs via Morlet wavelets

In [ ]:
frequencies = np.arange(7, 30, 3)
power = mne.time_frequency.tfr_morlet(aud_epochs, n_cycles=2, 
                                      return_itc=False, freqs=frequencies, decim=3)

In [ ]:
power.plot(picks=['EEG 021'])

### Construct and plot evoked responses from the epochs

In [ ]:
aud_evoked = aud_epochs.average()
vis_evoked = vis_epochs.average()

In [ ]:
aud_evoked.plot_joint(picks='eeg', times=[0., 0.08, 0.1, 0.12, 0.2])

In [ ]:
vis_evoked.plot_joint(picks='eeg', times=[0., 0.08, 0.1, 0.12, 0.2])

#### Plot topomaps

In [ ]:
vis_evoked.plot_topomap(times=[0., 0.08, 0.1, 0.12, 0.2], ch_type='eeg')

In [ ]:
# plot all sensors
vis_evoked.plot_topo(color='r', legend=False)

### Try different reference channels to see how they influence visual evoked responses 

In [ ]:
vis_evoked.plot_joint(picks='eeg')
vis_evoked.set_eeg_reference(ref_channels=['EEG 059']).plot_joint(picks='eeg')